In [ ]:
import time
import json
import operator
from typing import Annotated, List, TypedDict, Dict, Any

# Library AI & Graph
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

In [1]:
# --- DEFINISI STRUKTUR ---
class ScoringMultiAgent:
    def __init__(self, config: Dict[str, Any] = None):
        self.config = config or {}
        # Bisa ganti model di sini untuk eksperimen (misal: llama3, llama3.2)
        model_name = self.config.get("model", "llama3.2")
        
        print(f"🛠️ Initializing Agent with model: {model_name}")
        self.llm = ChatOllama(model=model_name, temperature=0, format="json")
        
        # Context Miskonsepsi (Bisa diedit-edit di sini untuk cari prompt terbaik)
        self.misconceptions_list = """
        1. Intentional Bug (IB)
        2. While Demon (WD)
        3. WhileIf / IfWhile
        4. Executed Once (EO)
        5. Drop Through Error (DT)
        6. Infinite Loop
        7. Wrong Order
        """
        self.app = self._build_graph()

    def _build_graph(self):
        # State Definition
        class AgentState(TypedDict):
            problem: str
            solution: str
            student_code: str
            rubric: str
            messages: Annotated[List[BaseMessage], operator.add]
            iteration_count: int
            final_json: Dict[str, Any]

        workflow = StateGraph(AgentState)
        workflow.add_node("Supervisor", self._supervisor_node)
        workflow.add_node("StyleChecker", self._style_checker_agent)
        workflow.add_node("LogicChecker", self._logic_checker_agent)
        workflow.set_entry_point("Supervisor")
        workflow.add_conditional_edges("Supervisor", self._router, 
                                     {"run_workers": "StyleChecker", END: END})
        workflow.add_edge("StyleChecker", "LogicChecker")
        workflow.add_edge("LogicChecker", "Supervisor")
        return workflow.compile()

    # --- NODE FUNCTIONS ---
    def _style_checker_agent(self, state):
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a Style Checker."),
            ("user", "Code:\n{student_code}\n\nReturn JSON with key 'style_analysis'.")
        ])
        chain = prompt | self.llm 
        res = chain.invoke({"student_code": state["student_code"]})
        return {"messages": [HumanMessage(content=f"Style: {res.content}")]}

    def _logic_checker_agent(self, state):
        prompt = ChatPromptTemplate.from_messages([
            ("system", f"You are a Logic Checker. Check misconceptions:\n{self.misconceptions_list}"),
            ("user", "Problem: {problem}\nSolution: {solution}\nStudent: {student_code}\n\nReturn JSON with key 'logic_analysis'.")
        ])
        chain = prompt | self.llm
        res = chain.invoke({
            "problem": state["problem"], "solution": state["solution"], 
            "student_code": state["student_code"]
        })
        return {"messages": [HumanMessage(content=f"Logic: {res.content}")]}

    def _supervisor_node(self, state):
        current_iter = state.get("iteration_count", 0)
        max_iter = self.config.get("max_iterations", 1)
        
        if current_iter >= max_iter:
            final_prompt = f"""
            Generate FINAL JSON.
            Format: {{"score": "0-100", "correct": "true/false", "summary": "text", "Misconceptions": "text"}}
            Rubric: {state['rubric']}
            Reports: {state['messages']}
            """
            try:
                res = self.llm.invoke(final_prompt)
                clean_json = res.content.replace("```json", "").replace("```", "").strip()
                final_dict = json.loads(clean_json)
            except:
                final_dict = {"error": "JSON Parse Fail", "raw": res.content}
            return {"final_json": final_dict, "iteration_count": current_iter + 1}
        
        return {"iteration_count": current_iter + 1}

    def _router(self, state):
        if state.get("final_json"): return END
        return "run_workers"

    def run(self, input_data):
        start_time = time.time()
        initial_state = {
            "problem": input_data.get("problem", ""),
            "solution": input_data.get("solution", ""),
            "student_code": input_data.get("pseudocode", ""),
            "rubric": input_data.get("rubric", ""),
            "messages": [],
            "iteration_count": 1
        }
        try:
            output = self.app.invoke(initial_state)
            final_json = output.get("final_json", {})
            summary_text = f"Score: {final_json.get('score')}. {final_json.get('summary')}"
        except Exception as e:
            final_json = {"error": str(e)}
            summary_text = "Error."
            
        return {
            'time': time.time() - start_time,
            'text': summary_text,
            'data': final_json
        }

NameError: name 'Dict' is not defined

In [ ]:
# 1. Konfigurasi
config = {
    "model": "llama3.2", # Coba ganti ke 'llama3' kalau mau bandingin hasil
    "max_iterations": 1
}

# 2. Inisialisasi
scorer = ScoringMultiAgent(config)

# 3. Test Case: Infinite Loop
inputs_fail = {
    "pseudocode": "x=1; WHILE x<5 DO; PRINT x; ENDWHILE", 
    "problem": "Cetak angka 1-5",
    "solution": "i=1; WHILE i<=5 DO; PRINT i; i=i+1; ENDWHILE",
    "rubric": "Benar=100"
}

# 4. Jalankan
print("⏳ Sedang menilai...")
result = scorer.run(inputs_fail)

# 5. Tampilkan Hasil
print("\n" + "="*40)
print(f"⏱️ Waktu: {result['time']:.4f} detik")
print("="*40)
print("📝 Summary Text:", result['text'])
print("📊 Data JSON:", json.dumps(result['data'], indent=2))